# Exploratory Data Analysis: Where Does Profit Disappear?

This dataset contains ~10,000 transactions from a retail operation. Before diving into specific analyses, I need to understand the data structure, identify patterns, and determine where the business is actually losing money.

The key columns we'll use:
- Sales: Revenue after discount is applied
- Quantity: Units sold per transaction
- Discount: Discount percentage given to customer
- Profit: Net profit/loss for that transaction (includes all costs)

The important thing to clarify: Profit already includes everything—product cost, shipping, operational expenses. We won't be able to break down individual cost components, but we don't need to. We need to understand the patterns.

## 01. Setup and Data Load

In [34]:
import pandas as pd
import numpy as np
import seaborn as sns
from scipy import stats
import matplotlib.pyplot as plt

pd.options.display.float_format = '{:.2f}'.format

In [35]:
df = pd.read_csv("data/Sample_Superstore.csv", encoding="latin-1")


## 02. Dataset Structure and First Rows

In [36]:
print(f"Shape: {df.shape}")
print(f"Columns: {df.columns.tolist()}")

Shape: (9994, 21)
Columns: ['Row ID', 'Order ID', 'Order Date', 'Ship Date', 'Ship Mode', 'Customer ID', 'Customer Name', 'Segment', 'Country', 'City', 'State', 'Postal Code', 'Region', 'Product ID', 'Category', 'Sub-Category', 'Product Name', 'Sales', 'Quantity', 'Discount', 'Profit']


In [37]:
# First 3 rows
df.head(3)

,Row ID,Order ID,Order Date,Ship Date,Ship Mode,Customer ID,Customer Name,Segment,Country,City,...,Postal Code,Region,Product ID,Category,Sub-Category,Product Name,Sales,Quantity,Discount,Profit
0,1,CA-2016-152156,11/8/2016,11/11/2016,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,...,42420,South,FUR-BO-10001798,Furniture,Bookcases,Bush Somerset Collection Bookcase,261.96,2,0.00,41.91
1,2,CA-2016-152156,11/8/2016,11/11/2016,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,...,42420,South,FUR-CH-10000454,Furniture,Chairs,"Hon Deluxe Fabric Upholstered Stacking Chairs,...",731.94,3,0.00,219.58
2,3,CA-2016-138688,6/12/2016,6/16/2016,Second Class,DV-13045,Darrin Van Huff,Corporate,United States,Los Angeles,...,90036,West,OFF-LA-10000240,Office Supplies,Labels,Self-Adhesive Address Labels for Typewriters b...,14.62,2,0.00,6.87


Mix of segments (Consumer, Corporate) and regions. The data shows transactions with varying profit levels. These first three have no discounts—all at full price.

## 03. Data Types and Column Information

In [38]:
df.info()

<class 'pandas.DataFrame'>
RangeIndex: 9994 entries, 0 to 9993
Data columns (total 21 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Row ID         9994 non-null   int64  
 1   Order ID       9994 non-null   str    
 2   Order Date     9994 non-null   str    
 3   Ship Date      9994 non-null   str    
 4   Ship Mode      9994 non-null   str    
 5   Customer ID    9994 non-null   str    
 6   Customer Name  9994 non-null   str    
 7   Segment        9994 non-null   str    
 8   Country        9994 non-null   str    
 9   City           9994 non-null   str    
 10  State          9994 non-null   str    
 11  Postal Code    9994 non-null   int64  
 12  Region         9994 non-null   str    
 13  Product ID     9994 non-null   str    
 14  Category       9994 non-null   str    
 15  Sub-Category   9994 non-null   str    
 16  Product Name   9994 non-null   str    
 17  Sales          9994 non-null   float64
 18  Quantity       9994

In [39]:
key_cols = ['Order Date', 'Ship Date', 'Sales', 'Quantity', 'Discount', 'Profit', 'Postal Code']
print(df[key_cols].dtypes)

Order Date         str
Ship Date          str
Sales          float64
Quantity         int64
Discount       float64
Profit         float64
Postal Code      int64
dtype: object


Order Date and Ship Date are loaded as object (string), need conversion to datetime. Postal Code is int64—should be string to preserve leading zeros. Sales, Discount, Profit are correct types.

## 04. Data Quality: Completeness and Duplicates

In [40]:
print("Missing values by column:")
print(df[key_cols].isnull().sum())

duplicates = df.duplicated().sum()
print(f"\nDuplicate rows: {duplicates}")

Missing values by column:
Order Date     0
Ship Date      0
Sales          0
Quantity       0
Discount       0
Profit         0
Postal Code    0
dtype: int64

Duplicate rows: 0


## 05. Categorical Data: Unique Values and Distribution

In [41]:
print("Segment distribution:")
print(df['Segment'].value_counts())

print("\nRegion distribution:")
print(df['Region'].value_counts())

print("\nCategory distribution:")
print(df['Category'].value_counts())

print(f"\nSub-Category unique values: {df['Sub-Category'].nunique()}")

Segment distribution:
Segment
Consumer       5191
Corporate      3020
Home Office    1783
Name: count, dtype: int64

Region distribution:
Region
West       3203
East       2848
Central    2323
South      1620
Name: count, dtype: int64

Category distribution:
Category
Office Supplies    6026
Furniture          2121
Technology         1847
Name: count, dtype: int64

Sub-Category unique values: 17


onsumer dominates (52% of transactions). West is largest region by volume. Office Supplies is dominant category (60% of transactions).

## 06. Numeric Data: Descriptive Statistics

In [42]:
print(df[['Sales', 'Quantity', 'Discount', 'Profit']].describe().to_string())

         Sales  Quantity  Discount   Profit
count  9994.00   9994.00   9994.00  9994.00
mean    229.86      3.79      0.16    28.66
std     623.25      2.23      0.21   234.26
min       0.44      1.00      0.00 -6599.98
25%      17.28      2.00      0.00     1.73
50%      54.49      3.00      0.20     8.67
75%     209.94      5.00      0.20    29.36
max   22638.48     14.00      0.80  8399.98


Sales range from $0.44 to $22,638 (wide range = outliers or bulk orders). Quantity is 1-5 units mostly. Discount median is 0% but goes up to 80%. Profit is the problem: from -$6,599 to +$8,399. Standard deviation of $234 on mean profit of $29 shows extreme variability.

## 07. Data Preparation: Type Conversions and Derived Metrics

In [43]:
df["Order Date"] = pd.to_datetime(df["Order Date"])
df["Ship Date"] = pd.to_datetime(df["Ship Date"])
df["Postal Code"] = df["Postal Code"].astype(str)

df['margin_pct'] = (df['Profit'] / df['Sales']) * 100
df['cost'] = df['Sales'] - df['Profit']

Converted dates to datetime for time-based analysis. Made Postal Code a string. Created margin_pct (will use throughout) and cost (since explicit cost column doesn't exist, calculated as Sales - Profit).

## 08. Sales and Profit Summary Statistics

In [44]:
total_sales = df['Sales'].sum()
total_profit = df['Profit'].sum()
profit_margin = (total_profit / total_sales) * 100

profitable_orders = (df['Profit'] > 0).sum()
losing_orders = (df['Profit'] <= 0).sum()

print(f"Total Sales: ${total_sales:,.2f}")
print(f"Total Profit: ${total_profit:,.2f}")
print(f"Profit Margin: {profit_margin:.2f}%")
print(f"\nOrders Breakdown:")
print(f"  Profitable orders: {profitable_orders:,} ({profitable_orders/len(df)*100:.1f}%)")
print(f"  Losing orders: {losing_orders:,} ({losing_orders/len(df)*100:.1f}%)")

Total Sales: $2,297,200.86
Total Profit: $286,397.02
Profit Margin: 12.47%

Orders Breakdown:
  Profitable orders: 8,058 (80.6%)
  Losing orders: 1,936 (19.4%)


12.5% profit margin is healthy for retail. But nearly 19% of orders are losing money. This means profitable orders are carrying unprofitable ones.

## 09. Margin Distribution Analysis

In [45]:
print("Margin % Distribution:")
print(f"Mean:         {df['margin_pct'].mean():.2f}%")
print(f"Median:       {df['margin_pct'].median():.2f}%")
print(f"Std Dev:      {df['margin_pct'].std():.2f}%")
print(f"\nPercentiles:")
print(f"10th:        {df['margin_pct'].quantile(0.10):.2f}%")
print(f"25th:        {df['margin_pct'].quantile(0.25):.2f}%")
print(f"75th:        {df['margin_pct'].quantile(0.75):.2f}%")
print(f"90th:        {df['margin_pct'].quantile(0.90):.2f}%")

Margin % Distribution:
Mean:         12.03%
Median:       27.00%
Std Dev:      46.68%

Percentiles:
10th:        -30.00%
25th:        7.50%
75th:        36.25%
90th:        47.00%


Median is higher than mean—extreme losses pull the average down. Margins range from deeply negative to 49%+. This variability is what we need to investigate.

## 10. Extreme Value Analysis: Top Losses and Gains

In [46]:
print("Worst 5 transactions:")
worst = df.nsmallest(10, 'Profit')[['Order ID', 'Sub-Category', 'Sales', 'Discount', 'Profit', 'margin_pct']]
print(worst.head(5).to_string())

Worst 5 transactions:
            Order ID Sub-Category   Sales  Discount   Profit  margin_pct
7772  CA-2016-108196     Machines 4499.98      0.70 -6599.98     -146.67
683   US-2017-168116     Machines 7999.98      0.50 -3839.99      -48.00
9774  CA-2014-169019      Binders 2177.58      0.80 -3701.89     -170.00
3011  CA-2017-134845     Machines 2549.99      0.70 -3399.98     -133.33
4991  US-2017-122714      Binders 1889.99      0.80 -2929.48     -155.00


In [47]:
print("Best 5 transactions:")
best = df.nlargest(10, 'Profit')[['Order ID', 'Category', 'Sub-Category', 'Sales', 'Discount', 'Profit', 'margin_pct']]
print(best.head(5).to_string())

Best 5 transactions:
            Order ID         Category Sub-Category    Sales  Discount  Profit  margin_pct
6826  CA-2016-118689       Technology      Copiers 17499.95      0.00 8399.98       48.00
8153  CA-2017-140151       Technology      Copiers 13999.96      0.00 6719.98       48.00
4190  CA-2017-166709       Technology      Copiers 10499.97      0.00 5039.99       48.00
9039  CA-2016-117121  Office Supplies      Binders  9892.74      0.00 4946.37       50.00
4098  CA-2014-116904  Office Supplies      Binders  9449.95      0.00 4630.48       49.00


Clear pattern: worst transactions have heavy discounts (70-80%) and huge losses. Best transactions have zero or minimal discounts and strong margins (48%). This points to a relationship between discount and profitability.

## 11. Discount Strategy Pattern

In [48]:
worst_10 = df.nsmallest(10, 'Profit')
best_10 = df.nlargest(10, 'Profit')

print("Worst 10 transactions - Average discount by product:")
print(worst_10[['Sub-Category', 'Discount']].groupby('Sub-Category').agg(['count', 'mean']).round(2))

print("\nBest 10 transactions - Average discount by product:")
print(best_10[['Sub-Category', 'Discount']].groupby('Sub-Category').agg(['count', 'mean']).round(2))

Worst 10 transactions - Average discount by product:
             Discount     
                count mean
Sub-Category              
Binders             4 0.80
Machines            5 0.62
Tables              1 0.40

Best 10 transactions - Average discount by product:
             Discount     
                count mean
Sub-Category              
Binders             4 0.00
Copiers             5 0.04
Machines            1 0.00


Binders show up in both worst and best lists. The difference is discount level, not the product. Binders at 0% are profitable; at 80% they destroy value. This suggests the problem isn't the product—it's the strategy.

## 12. Outlier Analysis

In [49]:
Q1 = df['margin_pct'].quantile(0.25)
Q3 = df['margin_pct'].quantile(0.75)
IQR = Q3 - Q1

outlier_low = Q1 - 1.5 * IQR
outlier_high = Q3 + 1.5 * IQR

extreme_low = (df['margin_pct'] < outlier_low).sum()
extreme_high = (df['margin_pct'] > outlier_high).sum()

print(f"Margin outliers using IQR method:")
print(f"Normal range: {outlier_low:.2f}% to {outlier_high:.2f}%")
print(f"Extreme losses (below {outlier_low:.2f}%): {extreme_low} transactions ({extreme_low/len(df)*100:.1f}%)")
print(f"Extreme gains (above {outlier_high:.2f}%): {extreme_high} transactions")

Margin outliers using IQR method:
Normal range: -35.62% to 79.38%
Extreme losses (below -35.62%): 952 transactions (9.5%)
Extreme gains (above 79.38%): 0 transactions


952 transactions (9.5%) are extreme losses. That's half of the 18.7% that are unprofitable. The losses don't spread evenly—they concentrate.

## 13. Summary: Initial Findings

From this exploration:

1. Overall profitability is solid (12.5% margin), but 18.7% of orders lose money. Profitable orders subsidize unprofitable ones.

2. Worst orders have heavy discounts (60-80%). Best orders have minimal or zero discounts (0-4%). Not a coincidence—there's a relationship.

3. Products themselves aren't inherently broken. Binders appear in both worst and best lists depending on discount. The problem is strategy, not product weakness.

4. Extreme losses concentrate in specific transactions. The problem is not distributed evenly.

5. Data spans 4 years with consistent volume. Complete with no missing values.

## 14. Next Analysis Steps

This establishes the baseline: we have a profitability problem concentrated in specific transactions with heavy discounts.

Next notebooks investigate the root causes:

- 02_discount_analysis: Do discounts actually drive volume? How much profit damage? Which regions and products are affected?
- 03_profit_analysis: What's the baseline profitability at full price? Which problems are discount-driven and fixable?

These will show whether the issue is discount strategy, weak product fundamentals, or both.


In [50]:
df.to_csv('sample_superstore_processed.csv', index=False)